In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler,MaxAbsScaler,MinMaxScaler,OrdinalEncoder
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
#%pip install catboost
import catboost as cb
from sklearn.model_selection import GridSearchCV

#from google.colab import drive
#drive.mount('/content/drive')
#cd drive/MyDrive/challange


In [7]:
train_station = pd.read_csv("data_to_use/train_onlyNext4EWM4andBack_EWM_remChar.csv", sep=",")
train_station['date'] = pd.to_datetime(train_station['date'])
train_station['Postcode'] = train_station['Postcode'].astype(str)
#train_station=train_station.loc[(train_station.date < '2021-02-1 00:00:00')]
#val 
station_val=pd.read_csv("data_to_use//train.csv", sep=",")
station_val['date'] = pd.to_datetime(station_val['date'])
station_val['Postcode'] = station_val['Postcode'].astype(str)

# test
test_station=pd.read_csv("data_to_use//test.csv", sep=",")
test_station['date'] = pd.to_datetime(test_station['date'])
test_station['Postcode'] = test_station['Postcode'].astype(str)


In [ ]:









train_station['hour'] = train_station.date.dt.hour
train_station['min'] = train_station.date.dt.minute
train_station['quarter'] = train_station.date.dt.quarter
train_station['dayofmonth'] = train_station.date.dt.day
train_station["labels"]=train_station.Available.astype(int).astype(str) + train_station.Charging.astype(int).astype(str) + train_station.Passive.astype(int).astype(str)+ train_station.Other.astype(int).astype(str)
test_station['hour'] = test_station.date.dt.hour
test_station['min'] = test_station.date.dt.minute
test_station['dayofmonth'] = test_station.date.dt.day


df_train=train_station.copy().loc[train_station.date < '2021-02-1 00:00:00']
df_train.loc[:,"labels"]=df_train.Available.astype(int).astype(str) + df_train.Charging.astype(int).astype(str) + df_train.Passive.astype(int).astype(str)+ df_train.Other.astype(int).astype(str)
df_val=station_val.copy().loc[station_val.date >= '2021-02-1 00:00:00']
df_val["labels"]=df_val.Available.astype(int).astype(str) + df_val.Charging.astype(str) + df_val.Passive.astype(str)+ df_val.Other.astype(int).astype(str)
df_val['hour'] = df_val.date.dt.hour
df_val['min'] = df_val.date.dt.minute
df_val['quarter'] = df_val.date.dt.quarter
df_val['dayofmonth'] = df_val.date.dt.day

In [ ]:
col_act=["Station","area","hour","dow","tod","trend"]

cat_x_train=train_station[col_act].copy()
cat_y_train=train_station.labels.copy()
cat_x_test=df_val[col_act].copy()
cat_y_test=df_val.labels.copy()
test_stationX=test_station[col_act].copy()


In [ ]:

cat_features_index = [0,1,2,3]
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.01, 0.1, 0.005],
         'l2_leaf_reg': [1,4,9],
         'iterations': [1500]}
#cb = cb.CatBoostClassifier(task_type="GPU",)
#cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3,n_jobs=-1,verbose=1)
#cb_model.fit(cat_x_train,cat_y_train, cat_features= cat_features_index,)

#print(cb_model.best_params_,cb_model.best_score_)
#'depth': 4, 'iterations': 100, 'l2_leaf_reg': 1, 'learning_rate': 0.01}
clf = cb.CatBoostClassifier(task_type="GPU",eval_metric="TotalF1",one_hot_max_size=250,loss_function="MultiClass",depth=4,
                            iterations= 1200, l2_leaf_reg= 1, learning_rate= 0.1)# 0.1
clf.fit(cat_x_train,cat_y_train, cat_features= cat_features_index,verbose=100)


0:	learn: 0.2566992	total: 80.3ms	remaining: 1m 36s
100:	learn: 0.3876387	total: 6.89s	remaining: 1m 14s
200:	learn: 0.4334583	total: 13.7s	remaining: 1m 8s
300:	learn: 0.4558929	total: 20.7s	remaining: 1m 1s
400:	learn: 0.4743862	total: 27.7s	remaining: 55.1s
500:	learn: 0.4902393	total: 34.8s	remaining: 48.5s
600:	learn: 0.5035740	total: 41.9s	remaining: 41.7s
700:	learn: 0.5145442	total: 48.9s	remaining: 34.8s
800:	learn: 0.5215834	total: 56s	remaining: 27.9s
900:	learn: 0.5290773	total: 1m 3s	remaining: 20.9s
1000:	learn: 0.5348358	total: 1m 10s	remaining: 13.9s
1100:	learn: 0.5393601	total: 1m 17s	remaining: 6.93s
1199:	learn: 0.5444499	total: 1m 24s	remaining: 0us


In [ ]:
y_pred_class=clf.predict(cat_x_test).ravel()

y_test_class = cat_y_test
nm_lab=np.unique(df_train["labels"])


print(classification_report(y_test_class, y_pred_class))
dict_ofdata={key: [int(value[0]),int(value[1]),int(value[2]),int(value[3])] for key,value in zip(nm_lab,nm_lab) }
df55=df_val[["Available","Charging","Passive","Other"]].copy()
df55[["Available_","Charging_","Passive_","Other_"]]=pd.DataFrame([dict_ofdata[i] for i in y_pred_class]).values
df55



dom=df_val[["Available","Charging","Passive","Other"]],pd.DataFrame([dict_ofdata[i] for i in y_pred_class])

mea00=MAE(dom[0],dom[1],multioutput="raw_values")
print(f'sklearn MAE \n{mea00}\n\n')
print(f'\n{mea00.mean()}\n\n')


print(f'sklearn MAE on the mean \n\n{100*mea00/dom[0].mean(axis=0)}%')
#0.5971670961120242
#0.49077350405879694
#0.47747026778729174
#0.46965412303430065
#sklearn MAE on the mean 
#0.3551931786373065
#0.4965369239271641
#0.49229151564305557
#0.48991939825015973
#0.4881465903952572
#0.2999956935841581 #last clean
#0.30238575437639514 remove charge
#Available     68.508524
#Charging     100.000000
#Passive      100.000000
#Other         88.296338
#dtype: float64%

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        0003       0.78      0.90      0.84     32923
        0012       0.80      0.66      0.72       921
        0021       0.83      0.16      0.26       408
        0030       0.00      0.00      0.00       163
        0102       0.53      0.11      0.19       319
        0111       0.43      0.01      0.02       245
        0120       0.00      0.00      0.00       143
        0201       0.57      0.05      0.09       732
        0210       0.00      0.00      0.00       365
        0300       0.00      0.00      0.00       506
        1002       0.88      0.79      0.83      3510
        1011       0.66      0.28      0.39      2060
        1020       0.52      0.17      0.26      3167
        1101       0.45      0.63      0.53      3258
        1110       0.00      0.00      0.00      3109
        1200       0.37      0.07      0.12      6479
        2001       0.52      0.35      0.42      3555
        2010       0.59    

In [ ]:
df55.tail(20)

,Available,Charging,Passive,Other,Available_,Charging_,Passive_,Other_
1851063,2,0,1,0,3,0,0,0
1851064,0,0,0,3,0,0,0,3
1851065,0,0,0,3,0,0,0,3
1851066,3,0,0,0,3,0,0,0
1851067,3,0,0,0,3,0,0,0
1851068,0,0,0,3,0,0,0,3
1851069,2,1,0,0,0,0,0,3
1851070,3,0,0,0,3,0,0,0
1851071,0,0,0,3,0,0,0,3
1851072,3,0,0,0,3,0,0,0


In [ ]:
test_stationX=test_station.copy()
test_pred_class=clf.predict(test_stationX[col_act]).ravel()
nm_lab=np.unique(train_station["labels"])
dict_ofdata={key: [int(value[0]),int(value[1]),int(value[2]),int(value[3])] for key,value in zip(nm_lab,nm_lab) }
df44=test_station.copy()
df44[["Available","Charging","Passive","Other"]]=pd.DataFrame([dict_ofdata[i] for i in test_pred_class]).values
df44

,date,Station,tod,dow,trend,Latitude,Longitude,Postcode,area,hour,min,dayofmonth,Available,Charging,Passive,Other
0,2021-02-19 00:00:00,FR*V75*EBELI*1*1,0,6,22203,48.85567,2.354089,75004,south,0,0,19,3,0,0,0
1,2021-02-19 00:00:00,FR*V75*EBELI*10*1,0,6,22203,48.86424,2.397724,75020,east,0,0,19,0,0,0,3
2,2021-02-19 00:00:00,FR*V75*EBELI*11*1,0,6,22203,48.86102,2.283080,75016,west,0,0,19,0,0,0,3
3,2021-02-19 00:00:00,FR*V75*EBELI*12*1,0,6,22203,48.82439,2.336857,75014,south,0,0,19,3,0,0,0
4,2021-02-19 00:00:00,FR*V75*EBELI*13*1,0,6,22203,48.88264,2.335728,75009,north,0,0,19,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165979,2021-03-10 23:45:00,FR*V75*EBELI*95*1,95,4,24122,48.86851,2.295469,75016,west,23,45,10,0,0,0,3
165980,2021-03-10 23:45:00,FR*V75*EBELI*96*1,95,4,24122,48.83550,2.349650,75013,south,23,45,10,3,0,0,0
165981,2021-03-10 23:45:00,FR*V75*EBELI*97*1,95,4,24122,48.83995,2.397822,75012,east,23,45,10,3,0,0,0
165982,2021-03-10 23:45:00,FR*V75*EBELI*98*1,95,4,24122,48.84998,2.361920,75004,south,23,45,10,3,0,0,0


In [ ]:
df44[['date', 'area', 'Station'] +
                   ["Available","Charging","Passive","Other"]].to_csv("data"+"/station.csv", index=False)